**Workshop**

จงหา best parameter ของ MACD  ที่ให้ผลตอบแทนเฉลี่ย มากที่สุด ของหุ้น Apple ใน 5 ปีที่ผ่านมา

## Import data

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib as ta

In [13]:
# define asset and date
ticker_name = 'AAPL'
start='2015-01-01' 
end='2021-01-01'

In [14]:
raw_df = yf.download(tickers=ticker_name, start=start, end=end)

[*********************100%***********************]  1 of 1 completed


## Optimization

In [15]:
df = raw_df.copy()

In [16]:
def macd_optimization(df, init_number=12, end_number=26, signal=9):
    all_trades_summary = pd.DataFrame(columns=['Average', 'Min', 'Max', 'i', 'j'])
    
    for i in range(init_number, end_number):
        for j in range(init_number+1, end_number+1):
            temp_df = df.copy()
            temp_df['macd'], temp_df['macdsignal'], temp_df['macdhist'] = ta.MACD(temp_df['Close'], fastperiod=i, slowperiod=j, signalperiod=signal)
            # entry signal sma12 > sma26 or macd > 0
            temp_df['signal'] = temp_df['macd'] > 0
            temp_df.ta.tsignals(temp_df.signal, asbool=True, append=True)

            # entry price signal
            temp_df['action_price'] = temp_df.Open.shift(-1)
            
            # result all transaction on i, j 
            trades_table = temp_df.iloc[:,-5:][temp_df.TS_Trades != 0] # Entry & Exit
            trades_table['return'] = trades_table['action_price'].pct_change() # pct_change() of Entry and Exit
            trades_summary = trades_table.loc[trades_table.TS_Exits == True] # real return each trade

            # create temp table for keep return on i, j and then insert into 'all_trades_summary' table
            a = trades_summary['return']
            temp_a = pd.DataFrame(data=[[a.mean(), a.min(), a.max(),i, j]], columns=['Average', 'Min', 'Max', 'i', 'j'])
            
            all_trades_summary = pd.concat([all_trades_summary, temp_a])
            all_trades_summary.dropna(inplace=True) # drop row is NaN (if i and j are same the output is NaN)

    return all_trades_summary

In [17]:
output_df = macd_optimization(df)

In [18]:
output_df

,Average,Min,Max,i,j
0,0.058208,-0.057952,0.710261,12,13
0,0.058074,-0.057952,0.710261,12,14
0,0.053687,-0.057952,0.638857,12,15
0,0.056950,-0.064051,0.690429,12,16
0,0.061573,-0.064051,0.690429,12,17
...,...,...,...,...,...
0,0.083710,-0.067374,0.572190,25,21
0,0.081834,-0.072509,0.572190,25,22
0,0.086434,-0.072253,0.659596,25,23
0,0.089524,-0.073168,0.548758,25,24


In [22]:
# best i (fast) and j (slow) line which get best average return
output_df.loc[output_df['Average'] == output_df.Average.max()]

,Average,Min,Max,i,j
0,0.102032,-0.080692,0.556565,25,26
